__Hoja de trabajo Proyecto BMW - Esther__

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


1.Importar dataset

In [4]:
bmw_original = pd.read_csv('dataset/bmw_pricing_v3.csv')

In [5]:
bmw_original.sample(5)

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
1776,BMW,116,155570.0,85.0,2012-10-01,diesel,black,hatchback,False,True,False,False,False,False,True,False,9900.0,2018-09-01
4640,BMW,X1,132592.0,120.0,NaN,diesel,white,suv,False,True,False,False,True,True,True,False,16100.0,2018-08-01
716,BMW,525,121332.0,160.0,2014-04-01,diesel,black,estate,True,True,False,NaN,True,False,True,True,26300.0,2018-03-01
1564,NaN,520,116761.0,135.0,2014-07-01,diesel,brown,NaN,True,False,False,NaN,True,False,True,True,20000.0,2018-07-01
2524,BMW,520,150004.0,120.0,2012-09-01,diesel,grey,NaN,True,True,False,NaN,False,False,True,False,13600.0,2018-01-01


In [6]:
bmw_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         